# Assignment  week 7

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: 

__Student id(s)__ : 

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

# Infoboxes

* We willen voor elke Wikipedia pagina de informatie uit de infobox halen en die gestructureerd opslaan.
* In het simpele geval is de infobox een lijst van attribuut-waarde paren:
    * het zijn attributen van de "entiteit" waar de pagina over gaat.
    * Dit zijn dus typisch _RDF subject-predicate-object triples_
    * Die je in eerste orde logica als een binaire relatie zou opslaan (bijvoorbeeld _DateOfBirth(Albert-Speer,'19 maart 1905')_)
* Je zal zien dat hier flink wat haken en ogen aan zitten.
    



## Inspecteer een voorbeeld

* We gaan eens kijken hoe we de attribuut-waarde paren uit de zogenaamde _infoboxen_ uit de Wikipedia paginas kunnen gaan halen.
* Vergelijk eens het begin van de file `NLWikipedia/small.xml` met de pagina <https://nl.wikipedia.org/wiki/Albert_Speer>
* Die infobox informatie ziet er goed en redelijk eenvoudig te extraheren uit.
* Er zijn verschillende _mediawiki_ (het formaat van wikipedia paginas) parsers in omloop. 

### Je bent niet de eerste met deze opdracht ....

* Zoek op het web.
* Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
* Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers
* Denk ook eens aan <http://pandoc.org/demos.html>
* Zie ook [WikipediaInfoBoxes](WikipediaInfoBoxes) is de assignments folder.

## Vragen

## Download, en extraheer.  (33%)
* Download de dump met alle NL wikipedia paginas.


1. Verzin een goede datastructuur voor de subject-predicate-object triples die je uit de infoboxes per pagina gaat extraheren.
1. Schrijf een functie die uit het text element het infobox deel extraheert, en die lijst attribuut-waarde paren omzet naar jouw datastructuur.
2. Pas je functie toe op het [voorbeeld](NLWikipedia/sample.xml, en debug je code.

### Nou op de hele file/dump
1. Zoek uit hoe je een file stromend kunt inlezen in Python.
2. Pas nu je code toe op de hele file. 
3. Misschien  moet je wel  steeds gegevens naar een file gaan wegschrijven. 
4. Is jouw gekozen data formaat  handig?

#### Hints:
* Wil niet meteen alles tegelijk. Als er te veel variatie in de data zit, richt je dan op de meest voorkomende gevallen, en probeer die goed te krijgen. 
* Probeer wel een beeld te krijgen van "wat je mist", en beschrijf dat ook. 

## Analyseer  (33%)

1. Maak statistieken, wat zijn de meest gebruikte attributen, hoe vaak worden ze gebruikt, hoeveel attributen zijn er? Welke waardes worden er veel gebruikt? Hoeveel attributen zie je gemiddeld/mediaan per pagina? 
2. Doe een analyse op de waardes van de attributen? Kan je bepaalde types vinden?
3. Doe een analyse op de uniformiteit van de attribuutnamen. Uit de literatuur blijkt dat er erg veel verschillende manieren worden gebruikt om hetzelfde attribuut weer te geven (bijv, sterfdag, sterfdatum, sterftejaar, etc). Is dat ook zo in jouw dataset? Hoe kom je daar eigenlijk achter? 
 
 
## Reflecteer  (33%)
* Je hebt nu flink wat tijd besteed aan wat de mensen achter DBpedia, Yago, FreeBase en meer van dit soort initiatieven ook al doen. De bedoeling van de opdracht was om te ervaren wat een klus het is om een kennisbank te vullen met "gestructureerde" informatie van het web. 
* Beschrijf puntsgewijs wat volgens jullie de lastige onderdelen zijn van dit proces.
* Licht er 2-3 uit, en vertel hoe jullie het probleem hebben aangepakt.

# Mijn antwoorden

In [8]:
#dict van dicts, {"page":{"Subject":_sub_, "Predicate":_pred_, "Obj":_obj_}}
import requests
from bs4 import BeautifulSoup
url = 'https://github.com/noracato/Kennissystemen/blob/master/Week7/small.xml'
r = requests.get(url) 
content = r.content
content = content.decode('UTF-8')
soup = BeautifulSoup(content, 'html')
text = soup.get_text()







Kennissystemen/small.xml at master · noracato/Kennissystemen · GitHub







































Skip to content






Sign up
Sign in




This repository






Explore


Features


Enterprise


Pricing
















    Watch
  

    2
  




    Star
  

      0
    




        Fork
      

      0
    




noracato/Kennissystemen














 Code

 


 Issues


 


 Pull requests


 





 Pulse

 


 Graphs

 




HTTPS clone URL








Subversion checkout URL







You can clone with
  HTTPS or Subversion.
  





                Download ZIP
              



Permalink




Branch:
master




Switch branches/tags









Branches


Tags









                master
              


Nothing to show




Nothing to show











Kennissystemen/Week7/small.xml





mlboon
Oct 13, 2015

added small.xml





1
           contributor
        



Users who have contributed to this file



mlboon








Raw
Blame
History









      9106 lines (7866 sloc